# AnyPlotter
This program allows you to plot 2d graphs (y against x), 2d scalar fields (in the form z= f(x,y)), 3d scalar fields, 2d vector fields, and 3d vector fields.

All functions must be written as though they were an equation written in python code.
For example:
    $$y=x^2 \to f(...)=x^{**}2$$
For vector fields, each dimension should be seperated by a comma. For example, a 2 dimensional vector field could be:
    $$f(...) = x^{**}2,y^{**}2$$
   

In [ ]:
import sys
sys.path.append('C:\\Users\\nikit\\AppData\\Local\\Programs\\Python\\python38\\lib\\site-packages')

import numpy as np
import ipywidgets as wd
from vpython import *
import parser
from PIL import Image
import sys
from IPython.display import clear_output
import random

CLEAR_QUAD = True
TESTING = False
im = Image.open('grad_colour.png') # Can be many different formats.
pix = im.load()           
c=pix[5,0]
def colourGrad(value, minValue, maxValue):
    s = 511-int(508*(int(value)-int(minValue))/(int(maxValue)-int(minValue)))
    try:
        c=pix[s,0]
    except:
        print(s)
        return vector(0,0,0)
    return vector(c[0],c[1],c[2])/255

def drawLabels(minV, maxV):
    Lt = label(pixel_pos=True, align='left',pos=vector(20, 20, 0), text=("Min Value: "+str("%.2f" % minV)))
    Ut = label(pixel_pos=True, align='left',pos=vector(20, scene.height-20, 0), text=("Max Value: "+str("%.2f" % maxV)))
    borBox = label(pixel_pos=True, align='left', pos=vector(20, 40 + (scene.height-20)/2, 0), height=scene.height-80, text="")
    for i in range(0,52):
        s= 511-int(508*(i/52))
        p=pix[s,0]
        c=vector(p[0],p[1],p[2])/255
        L = label(pixel_pos=True, pos=vector(20,40+i*(scene.height-80)/51,0),color=c, text="", border=False)
    

X_EQ=0
XY_EQ=1
XYZ_SCALER=2
XY_VEC=3
XYZ_VEC=4

def check(equation):
    allowed = [' ','x','y','z','.','1','2','3','4','5','6','7','8','9','0','*','/','+','-','^','e','(',')']
    allowedWords = ['cos', 'sec', 'sin','cosec', 'tan','cot', 'pi', 'sqrt', 'abs']
    char = list(equation)
    longW = max(len(x) for x in allowedWords)
    i=0
    while i < len(char):
        if(char[i] not in allowed):
            tryWords = [char[i]]
            allow=False
            for ii in range(1,longW):
                if(i+ii < len(char)):
                    nWord = tryWords[ii-1] + char[i+ii]
                    if(nWord in allowedWords):
                        i+= len(nWord)
                        allow=True
                        break
                    tryWords.append(nWord)
            if(allow):
                continue
            else:
                return False
        else:
            i+=1
    return True

inEq = str(input("Please enter your function in terms of x, y, and z: f(...) = ")) if not TESTING else "4*x,y**2,z+1"
finEq=[s.strip() for s in inEq.split(',')]

EQ_form = -1

x_=y_=z_=False
for p in finEq:
    if(check(p)==False):
        sys.exit("The equation ", p, " contains non valid characters")
    if(not x_):
        x_ = ('x' in p)
    if(not y_):
        y_ = ('y' in p)
    if(not x_):
        y_ = ('y' in p)
           
if(x_):
    if(y_):
        if(z_):
            if(len(finEq)==1):
                EQ_form=XYZ_SCALER
            elif(len(finEq)==3):
                EQ_form=XYZ_VEC
            else:
                print("Not valid!")
                sys.exit()
        else:
            if(len(finEq)==1):
                EQ_form=XY_EQ
            elif(len(finEq)==2):
                EQ_form=XY_VEC
            elif(len(finEq)==3):
                EQ_form=XYZ_VEC
            else:
                print("Not valid!")
                sys.exit()
    else:
        if(len(finEq)==1):
            EQ_form=X_EQ
        elif(len(finEq)==2):
            EQ_form=XY_VEC
        elif(len(finEq)==3):
                EQ_form=XYZ_VEC
        else:
            print("Not valid!")
            sys.exit()
    
print(EQ_form, flush=True)

#print(finEq)
e = np.e
pi = np.pi
def cos(a):
    return np.cos(a)
def sec(a):
    return 1/cos(a)

def sin(a):
    return np.sin(a)
def cosec(a):
    return 1/sin(a)

def tan(a):
    return 1/tan(a)
def cot(a):
    return 1/cos(a)

def sqrt(a):
    return np.sqrt(a)

def fxyz(x=0,y=0,z=0):
    return 0

code = None
if(len(finEq)==1):##Either an equation or scalar field
    finEq = finEq[0]
    code=parser.expr(finEq).compile()
    def fxyz_scal(x=0,y=0,z=0):
        return eval(code)
    fxyz=fxyz_scal
elif(len(finEq)==2):
    code = [parser.expr(finEq[0]).compile(),parser.expr(finEq[1]).compile()]
    
    def fxy_vec(x=0,y=0,z=0):
        return vector(eval(code[0]), eval(code[1]), 0)
    fxyz=fxy_vec
else:
    code = [parser.expr(finEq[0]).compile(),parser.expr(finEq[1]).compile(),parser.expr(finEq[2]).compile()]
    
    def fxyz_vec(x=0,y=0,z=0):
        return vector(eval(code[0]), eval(code[1]), eval(code[2]))
    fxyz=fxyz_vec

pointsPer1 = -1 if not TESTING else 2
while(pointsPer1==-1):
    try:
        pointsPer1 = int(input("Enter a resolution, the density of data points. Must be between 1 and 10: "))
        if( pointsPer1 not in range(1,10)):
            print("A resolution of ", pointsPer1, " is not valid.")
            pointsPer1 = -1
    except Exception as exep:
        pointsPer1=-1
        print("Not a valid resolution")
        print(exep)
        
resolution = 1.0/pointsPer1

def Bounds(whichBounds):
    if(TESTING):
        return [-2,2]
    st = ("Please enter the minimum and maximum values of " + whichBounds + ", seperated by a commar: ")
    while(True):
        try:
            b_ =str(input(st)).split(',')
        
            lower = float(b_[0])
            upper = float(b_[1])

            if(np.absolute(lower-upper)<resolution):
                print("A larger difference between minimum and maximum values of ", whichBounds, "is required.")
            elif(lower > upper):
                return [upper,lower]
            else:
                return [lower,upper]
        except Exception as exep:
            print("Bounds not valid")
            print(exep)

            
scene = canvas()


wholeSize = [[-1,1],[-1,1],[-1,1]]
xB= Bounds('x')
if(EQ_form==0):
    scene.autoscale = False
    scene.userzoom=False
    scene.autocenter=False
    xS = int((xB[1]-xB[0])/resolution)+1
    data=[None for x in range(xS)]
    for i in range(0, xS):
        x=i*resolution + xB[0]
        data[i]=vector(x, fxyz(x), 0.05)

    wholeSize[0] = [xB[0],xB[1]]
    wholeSize[1] = [min(yV.y for yV in data), max(yV.y for yV in data)]

    curv=curve(data[0], data[1], color=color.black, radius=0.1)
    for i in range(2, xS):
        curv.append(data[i])
    scene.background = color.white
    
    for xL in range(int(xB[0]-1), int(xB[1]+1)):
            cL = curve(vector(xL, wholeSize[1][0]-1, -0.05), vector(xL, wholeSize[1][1]+1, -0.05))
            if(xL % 5 ==0 ):
                xLabel = label(pos = vector(xL, wholeSize[1][0]-2, 0), text=str(xL))
    for yL in range(int(wholeSize[1][0]-1), int(wholeSize[1][1]+1)):
            cL = curve(vector(xB[0]-1, yL, -0.05), vector(xB[1]+1, yL, -0.05))
            if(yL % 5 ==0 ):
                yLabel = label(pos = vector(wholeSize[0][0]-2, yL, 0), text=str(yL))

elif(EQ_form==1):#3D plot where z=f(x,y)
    yB= Bounds('y')

    xS = int((xB[1]-xB[0])/resolution)+1
    yS = int((yB[1]-yB[0])/resolution)+1
    data = [[None for x in range(xS)] for y in range(yS)]
    wholeSize[0]=xB
    wholeSize[1]=yB
    needMinMax = True
    for i in range(0, xS):
        x=i*resolution + xB[0]
        for ii in range(0, yS):
            y=ii*resolution + yB[0]
            data[i][ii] = vector(x,y,fxyz(x,y))
            if(needMinMax):
                wholeSize[2] = [data[i][ii].z,data[i][ii].z]
                needMinMax=False
            elif(data[i][ii].z<wholeSize[2][0]):
                wholeSize[2][0]=data[i][ii].z
            elif(data[i][ii].z > wholeSize[2][1]):
                wholeSize[2][1] = data[i][ii].z
    print(wholeSize)
    #wholeSize=[xB,yB,zB]
    for i in range(0, xS-1):
        for ii in range(0, yS-1):
            N = cross(data[i+1][ii]-data[i][ii],data[i][ii+1]-data[i][ii]).norm()
            A=vertex(pos=data[i][ii], normal=N)
            B=vertex(pos=data[i+1][ii], normal=N)
            C=vertex(pos=data[i+1][ii+1], normal=N)
            D=vertex(pos=data[i][ii+1], normal=N)
            if(CLEAR_QUAD):
                col = colourGrad(random.random()*100, 0, 100)
                A.color = col
                B.color = col
                C.color = col
                D.color = col
            Q=quad(vs=[A,B,C,D])
elif(EQ_form==2): ##Scalar field in with x, y, z
    
    yB= Bounds('y')
    zB= Bounds('z')
    xS = int((xB[1]-xB[0])/resolution)+1
    yS = int((yB[1]-yB[0])/resolution)+1
    zS = int((zB[1]-zB[0])/resolution)+1
    data = [[[None for x in range(xS)] for y in range(yS)] for z in range(zS)]
    needMinMax = True
    minValue=0
    maxValue=0
    for i in range(0, xS):
        x=i*resolution + xB[0]
        for ii in range(0, yS):
            y=ii*resolution + yB[0]
            for iii in range(0,zS):
                z=iii*resolution + zB[0]                
                data[i][ii][iii] = fxyz(x,y,z)
                if(needMinMax):
                        minValue = data[i][ii][iii]
                        maxValye = minValue
                        needMinMax=False
                elif(data[i][ii][iii]<minValue):
                    minValue = data[i][ii][iii]
                elif(data[i][ii][iii]>maxValue):
                    maxValue = data[i][ii][iii]
    scene.lights = []
    scene.ambient=vector(0.6,0.6,0.6)
    wholeSize = [xB,yB,zB]
    for i in range(0, xS):
        x=i*resolution + xB[0]
        for ii in range(0, yS):
            y=ii*resolution + yB[0]
            for iii in range(0,zS):
                z=iii*resolution + zB[0]                
                s = resolution*0.25
                c=box(pos=vector(x,y,z), size=vector(resolution*0.25,resolution*0.25,resolution*0.25), color=colourGrad(data[i][ii][iii], minValue, maxValue))
    drawLabels(minValue, maxValue)
elif(EQ_form == 3): ##2D vector
    yB= Bounds('y')

    xS = int((xB[1]-xB[0])/resolution)+1
    yS = int((yB[1]-yB[0])/resolution)+1
    data = [[None for x in range(xS)] for y in range(yS)]
    wholeSize[0]=xB
    wholeSize[1]=yB
    needMinMax = True
    for i in range(0, xS):
        x=i*resolution + xB[0]
        for ii in range(0, yS):
            y=ii*resolution + yB[0]
            data[i][ii] = fxyz(x,y)
            if(needMinMax):
                wholeSize[2] = [mag(data[i][ii]),mag(data[i][ii])]
                needMinMax=False
            elif(mag(data[i][ii])<wholeSize[2][0]):
                wholeSize[2][0]=mag(data[i][ii])
            elif(mag(data[i][ii]) > wholeSize[2][1]):
                wholeSize[2][1] = mag(data[i][ii])
    for i in range(0, xS):
        x=i*resolution + xB[0]
        for ii in range(0, yS):
            y=ii*resolution + yB[0]
            col = colourGrad(mag(data[i][ii]),wholeSize[2][0],wholeSize[2][1])
            a=arrow(pos=vector(x,y,0), size=vector(resolution,resolution,resolution), axis=data[i][ii], color = col)

    drawLabels(wholeSize[2][0], wholeSize[2][1])
elif(EQ_form==4): #3D vector
    yB= Bounds('y')
    zB= Bounds('z')
    xS = int((xB[1]-xB[0])/resolution)+1
    yS = int((yB[1]-yB[0])/resolution)+1
    zS = int((zB[1]-zB[0])/resolution)+1
    wholeSize[0]=xB
    wholeSize[1]=yB
    wholeSize[2]=zB
    data = [[[None for x in range(xS)] for y in range(yS)] for z in range(zS)]
    magn =  [[[0 for x in range(xS)] for y in range(yS)] for z in range(zS)]
    needMinMax = True
    minValue=0
    maxValue=0

    for i in range(0, xS):
        x=i*resolution + xB[0]
        for ii in range(0, yS):
            y=ii*resolution + yB[0]
            for iii in range(0,zS):
                z=iii*resolution + zB[0]               
                data[i][ii][iii] = fxyz(x,y,z)
                magn[i][ii][iii] = mag(data[i][ii][iii])
                if(needMinMax):
                        minValue = magn[i][ii][iii]
                        maxValye = minValue
                        needMinMax=False
                elif(magn[i][ii][iii]<minValue):
                    minValue = magn[i][ii][iii]
                elif(magn[i][ii][iii]>maxValue):
                    maxValue = magn[i][ii][iii]
                    
    for i in range(0, xS):
        x=i*resolution + xB[0]
        for ii in range(0, yS):
            y=ii*resolution + yB[0]
            for iii in range(0,zS):
                z=iii*resolution + zB[0]                
                a=arrow(pos=vector(x,y,z), size=vector(resolution,resolution,resolution), axis=data[i][ii][iii], color = colourGrad(magn[i][ii][iii],minValue,maxValue))
        

    drawLabels(minValue, maxValue)
stop = wd.Button(description='Stop program')
test = wd.Button(description = 'Test button')
container = wd.HBox(children=[stop, test])
display(container)

running = True
hasLeft = False
def stop_handler(s):
    global running, hasLeft
    try:
        if(running):
            print("Closing the program", flush=True)
            running=False
            print("Press stop again to clear all", flush=True)
        elif(running==False and hasLeft):
            scene.delete()
            clear_output()
            %reset -f
    except:
        return None
        
stop.on_click(stop_handler)


test_obj=None
def addToTest(key, value):
    global test_obj
    if(type(key) == list):
        for i in range(0,len(key)):
            addToTest(key[i],value[i])
        return
    if(test_obj==None):
        test_obj = [[key,value]]
        return
    for t in test_obj:
        if(t[0]==key):
            t[1]=value
            return
    test_obj.append([key,value])

def test_handler(s):
    global test_obj
    if(test_obj==None):
        print("No test to print")
        return
    ii=0
    size = len(test_obj)
    print("Printing test values...")
    while ii < size:
        testOut = None
        for i in range (0,3):
            if(ii+i>=size):
                break
            else:
                if(testOut==None):
                    testOut = str(test_obj[ii+i][0]) + ": " + str(test_obj[ii+i][1])
                else:
                    testOut = testOut + ", " + str(test_obj[ii+i][0]) + ": " + str(test_obj[ii+i][1])
        print(testOut, flush=True)
        ii+=3

def allTestStr():
    global test_obj
    if(test_obj==None):
        print("No test to print")
        return
    ii=0
    size = len(test_obj)
    outStr = "Test values: \n"
    while ii < size:
        testOut = None
        for i in range (0,3):
            if(ii+i>=size):
                break
            else:
                if(testOut==None):
                    testOut = str(test_obj[ii+i][0]) + ": " + str(test_obj[ii+i][1])
                else:
                    testOut = testOut + ", " + str(test_obj[ii+i][0]) + ": " + str(test_obj[ii+i][1])
        outStr = outStr + testOut + "\n"
        ii+=3
    return outStr
        
        
test.on_click(test_handler)


middle = vector(wholeSize[0][0]+wholeSize[0][1],wholeSize[1][0]+wholeSize[1][1],wholeSize[2][0]+wholeSize[2][1]) * 0.5
camPos = middle + vector(0,0,np.absolute(wholeSize[0][0]-wholeSize[0][1])/2)
midPos = middle

def planeIntersect(startPos, direction, planeNormal, planePos):
    #return vector(0,0,0)
    
    
    bot = dot(planeNormal, direction)
    if(bot==0):
        return vector(0,0,0)
    
    s= dot(planeNormal, planePos-startPos)/bot
    return startPos + s*direction

def worldToPixel(worldPos, cameraPos=None, planePos=None):
    if(cameraPos==None):
        cameraPos = scene.camera.pos
    sceneRange = 0
    if(planePos==None):#Use plane at center point
        sceneRange = scene.range
    else:
        planeCenter = planeIntersect(cameraPos, norm(scene.camera.axis), planeNormal, planePos)
        sceneRange = mag(planeCenter-cameraPos) * np.tan(scene.fov/2)
    
    
    if(mag(scene.camera.axis)==0):
        return vector(0,0,0)
    
    horizontalAxis = norm(cross(scene.camera.axis, scene.up))
    worldWVal = dot(horizontalAxis, worldPos)
    camWVal = dot(horizontalAxis, cameraPos)

    vertAx = norm(cross(horizontalAxis,scene.camera.axis))
    worldHVal = dot(vertAx,worldPos)
    camHVal = dot(vertAx,cameraPos)
    
    rH=rW = sceneRange
    if(scene.width < scene.height):
        rH *= scene.height/scene.width
    elif(scene.height < scene.width):
        rW *= scene.width/scene.height
    
    
    pX = scene.width/2 * (1+ (worldWVal-camWVal)/rW)
    pY = scene.height/2 * (1+ (worldHVal-camHVal)/rH)
    
    return vector (pX,pY, 0)

def pixToWorld(pixelPos, cameraPos=None, planePos=None):
    if(cameraPos==None):
        cameraPos = scene.camera.pos
    sceneRange = 0
    planeCenter = None
    planeNormal = -norm(scene.camera.axis)
    if(planePos==None):#Use plane at center point
        sceneRange = scene.range
        planeCenter=scene.center
    else:
        planeCenter = planeIntersect(cameraPos, norm(scene.camera.axis), planeNormal, planePos)
        sceneRange = mag(planeCenter-cameraPos) * np.tan(scene.fov/2)
    
    rH=rW = sceneRange
    if(scene.width < scene.height):
        rH *= scene.height/scene.width
        #rH *= scene.width/scene.height
    elif(scene.height < scene.width):
        rW *= scene.width/scene.height
        #rH *= scene.width/scene.height
        #rW *= scene.height/scene.width
    
    #x and y distance from center of screen in terms of range
    relX = ((pixelPos.x - scene.width/2)/scene.width)*rW * 2
    relY = ((pixelPos.y - scene.height/2)/scene.height)*rH * 2
    
    horiz = norm(cross(scene.camera.axis, scene.up)) * relX
    vert = norm(cross(cross(scene.camera.axis, scene.up),scene.camera.axis)) * relY
    dep = mag(planeCenter-cameraPos) * norm(scene.camera.axis)
    #addToTest(['relX','relY','hor','vert','dep', 'sceneRange', 'range'],[relX/rW,relY/rH,horiz,vert,dep, sceneRange, scene.range])
    return (horiz+vert+dep) + cameraPos
    return vector(relX, 0, 0)
    #if(False):
        #horizontalAxis = norm(cross(scene.camera.axis, scene.up))
        #worldWVal = dot(horizontalAxis, worldPos)
        #camWVal = dot(horizontalAxis, cameraPos)
        #vertAx = norm(cross(horizontalAxis,scene.camera.axis))
        #worldHVal = dot(vertAx,worldPos)
        #camHVal = dot(vertAx,cameraPos)
    
    return vector(0,0,0)
    
    if(cameraPos==None):
        cameraPos = scene.camera.pos
    sceneRange = 0
    if(camToPlane==0):
        sceneRange=scene.range
    else:
        sceneRange = cameToPlane * np.tan(scene.fov/2)
        
    rH=rW = sceneRange
    
    if(scene.width < scene.height):
        rH *= scene.height/scene.width
    elif(scene.height < scene.width):
        rW *= scene.width/scene.height
    wX = (pixelPos.x*2/scene.width - 1)*rW + scene.camera.pos.x
    wY = (pixelPos.y*2/scene.height - 1)*rH + scene.camera.pos.y
    return vector(wX,wY,0)

def mouseScreenPos():
    
    planeNormal = -norm(scene.camera.axis)
    cent = scene.center
    mousePlanePos=None
    #addToTest(['camPos','camRay','planeNormal','screenCenter','range'], [scene.camera.pos, scene.mouse.ray, planeNormal, cent,scene.range])
    try:
        mousePlanePos = planeIntersect(scene.camera.pos, scene.mouse.ray, planeNormal, cent)
    except Exception as e:
        print("thisBit", flush=True)
        print(e, flush=True)
        addToTest("planExcept", e)

    return worldToPixel(mousePlanePos)
        

def getCameraDetails():
    global camPos, midPos
    return [camPos, midPos]

def Update():
    return 0

scene.autoscale=False
scene.userzoom=False
#Set up coord systems:
if(EQ_form==X_EQ or EQ_form ==XY_VEC):
    
    starVec = vector(0.5*(wholeSize[0][0]+wholeSize[0][1]), 0.5*(wholeSize[1][0]+wholeSize[1][1]), 1)
    minVec = vector(wholeSize[0][0],wholeSize[1][0], 1)
    maxVec = vector(wholeSize[0][1],wholeSize[1][1], 10)
    widthVec = maxVec-minVec
    #starVec.z = max([wholeSize[0][1]-starVec.x,starVec.y])

    xPos = wd.FloatSlider(description='X position', min=minVec.x, max=maxVec.x, step=2*resolution, value=starVec.x)
    yPos = wd.FloatSlider(description='Y position', min=minVec.y, max=maxVec.y, step=2*resolution, value=starVec.y)
    zoom = wd.FloatSlider(description='Zoom', min=minVec.z, max=maxVec.z, step=0.5, value=starVec.z)
    display(wd.VBox(children=[xPos,yPos, zoom]))
    
    mouseScreenPosLabel = label(pixel_pos = True, align='right', pos=vector(scene.width-20, scene.height-20, 0), text="Mouse position: (0,0)", box=False)
    
    
    xArrow = arrow(pos=vector(0,0,0), axis=vector(1,0,0), color=color.red)
    xArrowLabel = label(pos=vector(0,0,0), text="X", box=False)
    yArrow = arrow(pos=vector(0,0,0), axis=vector(0,1,0), color=color.red)
    yArrowLabel = label(pos=vector(0,0,0), text="Y", box=False)
    
    clickPos = None
    clickNo = 0
    scrPos = None
    def onClick():
        global clickPos, clickNo, scrPos
        clickNo += 1
        clickPos = planeIntersect(scene.camera.pos, scene.mouse.ray, vector(0,0,-1), vector(0,0,0))
        scrPos = mouseScreenPos()

    scene.bind('mousedown',onClick)
    
    def Update2D():
        scene.range=max(widthVec.x,widthVec.y)*((maxVec.z+1-zoom.value)/10)

        scene.camera.pos=vector(xPos.value,yPos.value,scene.camera.pos.z)
        mousePosZ0 = planeIntersect(scene.camera.pos, scene.mouse.ray, vector(0,0,-1), vector(0,0,0))
        mouseScreenPosLabel.text = str("Mouse position: ({0:.2f},{1:.2f})".format(mousePosZ0.x,mousePosZ0.y))
        
    
        xArrow.pos = pixToWorld(vector(50,50,0))
        yArrow.pos = pixToWorld(vector(50,50,0))
        arLen = abs((pixToWorld(vector(0,0,0))-pixToWorld(vector(50,0,0))).x)
        xArrow.length = arLen
        yArrow.length = arLen
        xArrowLabel.pos=xArrow.pos + vector(xArrow.length/2,0,0)
        yArrowLabel.pos=yArrow.pos + vector(0,yArrow.length/2,0)
        
    
    if(EQ_form==X_EQ):
        mouseClickLabel = label(pixel_pos = True, align='right', pos=vector(scene.width-20, scene.height-40, 0), text="f(x) = " + str(finEq) + "\nClick the screen to find values at a point", box=False)

        def X_EQ_Update():
            global clickPos, clickNo
            Update2D()
            
            if(clickPos!=None):                
                la = label(pixel_pos=True, align='right', pos=vector(scene.width-20, scene.height-60-clickNo*20, 0), box=False, text="<{0:.2f},{1:.2f}>".format(clickPos.x,fxyz(x=clickPos.x)))
                clickPos=None  
        
        Update = X_EQ_Update
        
    elif(EQ_form==XY_VEC):
        mouseClickLabel = label(pixel_pos = True, align='right', pos=vector(scene.width-20, scene.height-40, 0), text="G(x,y) = " + str(finEq) + "\nClick the screen to find values at a point", box=False)
        def XY_VEC_Update():
            global clickPos, clickNo
            Update2D()
            
            if(clickPos!=None):
                fVal = fxyz(x=clickPos.x, y=clickPos.y)
                la = label(pixel_pos=True, align='right', pos=vector(scene.width-20, scene.height-60-clickNo*20, 0), box=False, text="<{0:.2f},{1:.2f}>=<{2:.1f},{3:.1f}>".format(clickPos.x,clickPos.y, fVal.x,fVal.y))
                clickPos=None

        Update = XY_VEC_Update
        
else:
    scene.autoscale=False
    scene.userzoom=False
    scene.up=vector(0,0,1)
    
    
    startVec = vector(wholeSize[0][0]+wholeSize[0][1], wholeSize[1][0]+wholeSize[1][1], wholeSize[2][0]+wholeSize[2][1])/2
    minVec = vector(wholeSize[0][0],wholeSize[1][0], wholeSize[2][0]) - 2*vector(abs(wholeSize[0][0]),abs(wholeSize[1][0]), abs(wholeSize[2][0]))
    maxVec = vector(wholeSize[0][1],wholeSize[1][1], wholeSize[2][1]) + 2*vector(abs(wholeSize[0][0]),abs(wholeSize[1][0]), abs(wholeSize[2][0]))
    widthVec = maxVec-minVec
    
    scene.camera.axis=vector(0,maxVec.y,0)
    
    xPos = wd.FloatSlider(description='X: ', min=minVec.x, max =maxVec.x, step=2*resolution, value=startVec.x)
    yPos = wd.FloatSlider(description='Y: ', min=minVec.y, max =maxVec.y, step=2*resolution, value=startVec.y)
    zPos = wd.FloatSlider(description='Z: ', min=minVec.z, max =maxVec.z, step=2*resolution, value=startVec.z)
    
    sliderBox = wd.VBox(children=[xPos,yPos,zPos]) 
    
    coordinatesSystem = wd.Dropdown(options=['cartesian', 'cylindrical', 'spherical'], value='cartesian',
                       description='Coordinate system')
    display(wd.HBox([coordinatesSystem,sliderBox]))
    currentCoords = coordinatesSystem.value

    cartesianCoords=vector(xPos.value, yPos.value, zPos.value)             
    
    axisReference = [arrow(axis=vector(1,0,0)),label(text="X", box=False),
                     arrow(axis=vector(0,1,0)),label(text="Y", box=False),
                     arrow(axis=vector(0,0,1)),label(text="Z", box=False),
                     arrow(axis=vector(-1,0,0), visible=False, my_id='phi'), label(text="phi", visible=False, my_id='phi', box=False), curve(my_id='phi', visible=False), curve(my_id='ref', visible=False),
                     arrow(axis=vector(0,0,-1), visible=False, my_id='theta'), label(text="theta", visible=False, my_id='theta', box=False), curve(my_id='theta', visible=False), curve(my_id='ref', visible=False)]
    
    for i in range(0,10):
        theta = np.pi/2 * i/9
        axisReference[8].append(vector(np.cos(theta),np.sin(theta),0))
        axisReference[9].append(vector(np.cos(theta),np.sin(theta),0))
        axisReference[12].append(vector(0,np.sin(theta),np.cos(theta)))
        axisReference[13].append(vector(0,np.sin(theta),np.cos(theta)))
        
    def updateAxisReference():
        pL = scene.camera.pos + scene.camera.axis
        axPos = pixToWorld(vector(75,75,0))
        axLen = mag(pixToWorld(vector(0,0,0))-pixToWorld(vector(75,0,0)))
        #axLen = 1
        for i in range(0,len(axisReference)):
            if(axisReference[i].visible == False):
                continue
                
            refID = None
            try:
                refID=axisReference[i].my_id
            except:
                refID=None
            
            if(type(axisReference[i])==arrow):
                if(refID == None):
                    axisReference[i].pos = axPos
                    axisReference[i].length = axLen
                    addToTest('ArrowPos_'+ str(i), axisReference[i].pos)
                else:
                    axLen_ = axLen * 0.2
                    if(refID=='phi'):
                        axisReference[i].pos = axPos + vector(axLen_, axLen-axLen_*0.2, 0)
                        axisReference[i].axis = vector(-1,0.2*axLen_,0)
                    elif(refID=='theta'):
                        axisReference[i].pos = axPos + vector(0, axLen-axLen_*0.2, axLen_)
                        axisReference[i].axis = vector(0,0.2*axLen_,-1)
                    axisReference[i].length = axLen_
                    axisReference[i].shaftwidth= 0.2 * axLen_
            elif(type(axisReference[i])==label):
                if(refID==None):
                    axisReference[i].pos = axPos + axLen * norm(axisReference[i-1].axis)/2
                elif(refID=='phi'):
                    axisReference[i].pos = axPos + axLen * 0.707 * vector(1,1,0)
                elif(refID=='theta'):
                    axisReference[i].pos = axPos + axLen * 0.707 * vector(0,1,1)
            elif(type(axisReference[i])==curve):
                if(refID!='ref'):
                    #axisReference[i].radius = axLen*0.1
                    for p in range(0,10):
                        axisReference[i].modify(p, pos = axPos + axLen*norm(axisReference[i+1].point(p)['pos'])) 
                        
    def coordSys_handler(s):
        global xPos,yPos,zPos,currentCoords, cartesianCoords, axisReference
        currentCoords=s.new
        #print(s.old, flush=True)
        for i in range(6, len(axisReference)):
            axisReference[i].visible=False
        
        if(s.new=='cartesian'):
            xPos.description="X: "
            yPos.description="Y: "
            zPos.description="Z: "
            
            xPos.min = minVec.x
            xPos.max = maxVec.x
            yPos.min = minVec.y
            yPos.max = maxVec.y
            zPos.min = minVec.z
            zPos.max = maxVec.z
            
            xPos.value =cartesianCoords.x
            yPos.value =cartesianCoords.y
            zPos.value =cartesianCoords.z
        elif(s.new=='cylindrical'):
            axisReference[6].visible=axisReference[7].visible=axisReference[8].visible=True
            xPos.description="\(\\rho\): "
            yPos.description="\(\phi\): "
            zPos.description="Z: "
            
            xPos.min = 0
            xPos.max = 2*np.sqrt(maxVec.x**2 + maxVec.y**2)
            yPos.min = -180
            yPos.max = 180
            zPos.min = minVec.z
            zPos.max = maxVec.z
            
            xPos.value = np.sqrt(cartesianCoords.x**2 + cartesianCoords.y**2)
            #addToTest('cart x', cartesianCoords.x)
            yPos.value = 90
            if(cartesianCoords.x!=0):
                yPos.value=np.rad2deg(np.arctan(cartesianCoords.y/cartesianCoords.x))
            zPos.value = cartesianCoords.z

        elif(s.new=='spherical'):
            axisReference[6].visible=axisReference[7].visible=axisReference[8].visible=True
            axisReference[10].visible=axisReference[11].visible=axisReference[12].visible=True
            xPos.description="r: "
            yPos.description="\(\\theta\): "
            zPos.description="\(\\rho\): "
            
            xPos.min = 0
            xPos.max = 2*mag(maxVec)
            yPos.min = 0
            yPos.max = 180
            zPos.min = -180
            zPos.max = 180
            
            xPos.value = mag(cartesianCoords)
            yPos.value = 90
            if(cartesianCoords.z!=0):
                yPos.value = np.rad2deg(np.arctan(np.sqrt(cartesianCoords.x**2 + cartesianCoords.y**2)/cartesianCoords.z))
            zPos.value = 90
            if(cartesianCoords.x != 0):
                zPos.value = np.rad2deg(np.arctan(cartesianCoords.y/cartesianCoords.x))
                
    coordinatesSystem.observe(coordSys_handler, names='value')
    
    def updateCoords():
        if(currentCoords=='cartesian'):
            cartesianCoords.x = xPos.value
            cartesianCoords.y = yPos.value
            cartesianCoords.z = zPos.value
            scene.camera.axis = vector(0, max(1,-yPos.value), 0)
        elif(currentCoords=='cylindrical'):
            cartesianCoords.x = -xPos.value * np.cos(np.deg2rad(yPos.value))
            cartesianCoords.y = -xPos.value * np.sin(np.deg2rad(yPos.value))
            cartesianCoords.z = zPos.value
            scene.camera.axis = max(xPos.value,0.01) * vector(np.cos(np.deg2rad(yPos.value)),np.sin(np.deg2rad(yPos.value)),0)
            
        elif(currentCoords=='spherical'):
            cartesianCoords.x = -xPos.value * np.sin(np.deg2rad(yPos.value+0.1))*np.cos(np.deg2rad(zPos.value))
            cartesianCoords.y = -xPos.value * np.sin(np.deg2rad(yPos.value+0.1))*np.sin(np.deg2rad(zPos.value))
            cartesianCoords.z = xPos.value * np.cos(np.deg2rad(yPos.value+0.1))
            scene.camera.axis = max(xPos.max,0.01) * vector(np.sin(np.deg2rad(yPos.value+0.1))*np.cos(np.deg2rad(zPos.value)),np.sin(np.deg2rad(yPos.value+0.1))*np.sin(np.deg2rad(zPos.value)),-np.cos(np.deg2rad(yPos.value+0.1)))
        
    #xArrow.pos = pixToWorld(vector(50,50,0))
    #yArrow.pos = pixToWorld(vector(50,50,0))
    #arLen = abs((pixToWorld(vector(0,0,0))-pixToWorld(vector(50,0,0))).x)
    #xArrow.length = arLen
    #yArrow.length = arLen
    #xArrowLabel.pos=xArrow.pos + vector(xArrow.length/2,0,0)
    #yArrowLabel.pos=yArrow.pos + vector(0,yArrow.length/2,0)
    #axisReference = [arrow(pos=vector(0,0,0), axis=vector(1,0,0)),label(pos=vector(0,0,0), text="X"),
    #                 arrow(pos=vector(0,0,0),axis=vector(0,1,0)),label(pos=vector(0,0,0), text="Y"),
    #                 arrow(pos=vector(0,0,0),axis=vector(0,0,1)),label(pos=vector(0,0,0), text="Z")]
    
    mouseScreenPosLabel = label(pixel_pos = True, align='right', pos=vector(scene.width-20, scene.height-20, 0), text="Mouse position: (0,0)", box=False)
    def Update3D():
        updateCoords()
        scene.camera.pos=cartesianCoords
        updateAxisReference()
        mousePosZ0 = mouseScreenPos()
        mouseScreenPosLabel.text = str("Mouse position: ({0:.0f},{1:.0f})".format(mousePosZ0.x,mousePosZ0.y))
        
    
    clickObj = None
    def onClick():
        global clickObj, clickNo
        hit = scene.mouse.pick
        #test_handler(None)
        if(hit != None):
            clickObj = hit
        #addToTest(['cartCoords', 'axis', 'camPos', 'center', 'mouseScr', 'ray', 'range'],[cartesianCoords, scene.camera.axis, scene.camera.pos, scene.center, planT,scene.mouse.ray, scene.range])
    scene.bind('mousedown',onClick)

    Update=Update3D
    
    if(EQ_form == XY_EQ):
        mouseClickLabel = label(pixel_pos = True, align='right', pos=vector(scene.width-20, scene.height-40, 0), text="f(x,y) = " + finEq + "\nClick the screen to find values at a point", box=False)
        clickNo = 0
        mouseScreenPosLabel = label(pixel_pos = True, align='right', pos=vector(scene.width-20,scene.height-100,0), text="Mouse screen pos")
        pixToWorldLabel = label(pixel_pos = True, align='right', pos=vector(scene.width-20,scene.height-120,0), text="Mouse screen to world")
        allTestStringLabel = label(pixel_pos = True, align='right', pos=vector(scene.width-20,scene.height-140,0), text="")
        def XY_EQ_update():
            global clickObj, clickNo, pixToWorld
            Update3D()
            
            if(clickObj != None and type(clickObj)==quad):
                clickPos = (clickObj.vs[0].pos + clickObj.vs[1].pos + clickObj.vs[2].pos + clickObj.vs[3].pos)/4
                clickNo += 1
                
                la = label(pixel_pos=True, align='right', pos=vector(scene.width-20, scene.height-60-clickNo*20, 0), box=False, text="f({0:.2f},{1:.2f}) = {2:.1f}".format(clickPos.x,clickPos.y, fxyz(x=clickPos.x, y=clickPos.y)))
                clickObj=None
            
                
        Update = XY_EQ_update
        
Update()
scene.waitfor("redraw")
while running:
    rate(50)
    Update()
hasLeft=True

Please enter your function in terms of x, y, and z: f(...) = x**2 + y**2
1
Enter a resolution, the density of data points. Must be between 1 and 10: 5


<IPython.core.display.Javascript object>

Please enter the minimum and maximum values of x, seperated by a commar: -5,5
Please enter the minimum and maximum values of y, seperated by a commar: -5,5
[[-5.0, 5.0], [-5.0, 5.0], [0.0, 50.0]]
